- Importing Files and Setting Model Path

In [28]:
import torch
import torch.nn as nn

# Specify the path to the .pth file
model_path = "M1_Task1.pth"

- Define the model

In [29]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output, _ = self.rnn(x)
        output = self.fc(output)
        return output

In [30]:
# Initialize the model
INPUT_SIZE = 384 # Dimension of the input embeddings
HIDDEN_SIZE = 128 # Dimension of the hidden state
OUTPUT_SIZE = 8 # Number of classes

# Load the saved model
model = RNNModel(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)
model.load_state_dict(torch.load("M1_Task1.pth"))

<All keys matched successfully>

In [31]:
import pickle
# Load these embeddings from the pickle files
val_embeddings_file = "val_embeddings.pkl"
val_embeddings_loaded = pickle.load(open(val_embeddings_file, "rb"))

- Load Labels (Emotions and Speakers) File

In [32]:
val_labels_file = "task1_labels_dev.pkl"
val_labels_loaded = pickle.load(open(val_labels_file, "rb"))

val_speaker_ids_file = "speakers_dev.pkl"
val_speaker_ids_loaded = pickle.load(open(val_speaker_ids_file, "rb"))

- Dictionary for all unique Speakers in the Dataset

In [33]:
# Get all unique speakers
unique_speakers = set()
for episode_id in val_speaker_ids_loaded:
    unique_speakers.update(val_speaker_ids_loaded[episode_id])

print(unique_speakers)
# Create separate dictionaries for each speaker to store predicted emotions
speaker_predictions_dict = {speaker: {} for speaker in unique_speakers}

{'Mike', 'Kathy', 'Lauren', 'Leslie', 'an', 'The Fireman', 'Student', 'Eric', 'The Head Librarian', 'Joey', 'Stevens', 'Receptionist', 'Larry', 'Guy', '1st Customer', 'Tony', 'Nurse', 'Tom', 'Bobby', 'Carol', 'Dr. Zane', 'Fireman No. 3', 'Phoebe Sr', 'Stranger', 'The Museum Official', 'Joey and Chandler', 'Jester', 'Dr. Wesley', 'Monica', 'Ross', 'The Guys', 'Stage Manager', 'Girl', 'Mrs. Green', 'A Student', 'Young Ethan', 'Lydia', 'Chandler', 'Kristin', 'Wayne', 'The Director', 'Carl', 'Bernice', 'Kate', 'Susan', 'Kim', 'Emily', 'Mark', 'All', 'The Woman', 'Paul', 'Phoebe and Rachel', 'Boy in the Cape', 'Isabella', '3rd Customer', 'Barry', 'Stage Director', 'Chloe', 'Dina', 'Director', 'Jill', 'Kristen', 'Richard', 'Mischa', 'Phoebe Sr.', 'Bonnie', 'Dr. Oberman', 'Duncan', 'Jade', 'Hoshi', 'Chip', 'Mrs. Tedlock', 'Both', 'Dana', 'Ursula', 'Professore Clerk', 'Krista', 'Janice', 'Teacher', 'Mindy', 'Ms. McKenna', 'Charlton Heston', 'Mr. Posner', 'Terry', 'Woman', 'David', 'Aunt Lillia

In [34]:
# Define a function to predict emotions for a given set of embeddings
def predict_emotions(embeddings):
    model.eval()
    #print(embeddings.shape)
    with torch.no_grad():
        # Convert the embeddings to a tensor
        embeddings_tensor = torch.tensor(embeddings).unsqueeze(0)
        # Get the model predictions
        output = model(embeddings_tensor)
        predictions = output.argmax(dim=-1).squeeze().tolist()
        # print(len(predictions))
    return predictions

In [35]:
# Predict emotions for each utterance and associate with each speaker
for episode_id, embeddings in val_embeddings_loaded.items():
    predictions = predict_emotions(embeddings)
    speakers = val_speaker_ids_loaded[episode_id]
    for speaker in speakers:
        # Filter predictions for the current speaker
        speaker_predictions = [pred for pred, spk in zip(predictions, val_speaker_ids_loaded[episode_id]) if spk == speaker]
        speaker_predictions_dict[speaker][episode_id] = speaker_predictions

for speaker in speaker_predictions_dict:
    print(f"Speaker: {speaker}")
    for episode_id, predictions in speaker_predictions_dict[speaker].items():
        print(f"Episode: {episode_id}, Predictions: {predictions}")
    print()


Speaker: Mike
Episode: utterance_1666, Predictions: [7]
Episode: utterance_1664, Predictions: [7]
Episode: utterance_3649, Predictions: [7, 7, 7, 7]

Speaker: Kathy
Episode: utterance_1158, Predictions: [5, 2]

Speaker: Lauren
Episode: utterance_211, Predictions: [7, 4]

Speaker: Leslie
Episode: utterance_701, Predictions: [2, 2, 1]
Episode: utterance_699, Predictions: [2, 2]

Speaker: an
Episode: utterance_1606, Predictions: [2, 2]

Speaker: The Fireman
Episode: utterance_1798, Predictions: [4, 7]
Episode: utterance_1797, Predictions: [4]

Speaker: Student
Episode: utterance_1429, Predictions: [7]
Episode: utterance_1430, Predictions: [7, 7]
Episode: utterance_3490, Predictions: [7, 7]

Speaker: Eric
Episode: utterance_3496, Predictions: [3, 1, 1, 3, 1, 1]
Episode: utterance_2177, Predictions: [7, 5]

Speaker: The Head Librarian
Episode: utterance_2699, Predictions: [7, 7]

Speaker: Joey
Episode: utterance_3421, Predictions: [7, 5, 4, 7, 7, 7, 3, 7, 1, 7]
Episode: utterance_172, Predi

In [36]:
from sklearn.metrics import f1_score

# Encode the labels by a fixed mapping
mapping = {
    "-1": 0,
    "sadness": 1,
    "joy": 2,
    "fear": 3,
    "anger": 4,
    "surprise": 5,
    "disgust": 6,
    "neutral": 7
}

# Function to detect emotion flips for a given sequence of emotions
def detect_emotion_flips(predicted_emotions, true_emotions):
    valid_flips = []
    invalid_flips = []
    for i in range(1, len(predicted_emotions)):
        if predicted_emotions[i] != predicted_emotions[i - 1]:
            if predicted_emotions[i-1] == true_emotions[i-1] and predicted_emotions[i] == true_emotions[i]:
                valid_flips.append((i - 1, i))  # Store the indices of the flip
            else:
                invalid_flips.append((i - 1, i))
            
    return valid_flips, invalid_flips

In [37]:
def totalnum_flips(true_emotions):
    total_flips = 0
    for i in range(1, len(true_emotions)):
        if true_emotions[i] != true_emotions[i - 1]:
            total_flips += 1
    return total_flips
    

In [38]:
import numpy as np

# Calculate emotion flips and metrics for each speaker
speaker_metrics = {}
final=0
final_correct=0

for speaker in unique_speakers:
    valid_flips = []
    invalid_flips = []
    true_labels = []
    predicted_labels = []
    total_flips = 0
    print(f"Speaker: {speaker}")
    
    for episode_id, true_emotions in val_labels_loaded.items():
        if speaker in val_speaker_ids_loaded[episode_id]:
            print(f"Episode: {episode_id}")
            predicted_emotions = speaker_predictions_dict[speaker][episode_id]
            #convert the emotions to the encoded values
            true_emotions = [mapping[emotion] for emotion, spk in zip(true_emotions, val_speaker_ids_loaded[episode_id]) if spk == speaker]
            print(f"True Emotions: {true_emotions}")
            print(f"Predicted Emotions: {predicted_emotions}")
            valid, invalid = detect_emotion_flips(predicted_emotions, true_emotions)
            total_flips += totalnum_flips(true_emotions)
            valid_flips.extend(valid)
            invalid_flips.extend(invalid)
            true_labels.extend(true_emotions)
            predicted_labels.extend(predicted_emotions)
    
    # Calculate accuracy of valid flips
    correct_flips = len(valid_flips)
    final+=total_flips
    final_correct+=correct_flips
    accuracy = correct_flips / total_flips if total_flips > 0 else 0
    
    # Calculate macro F1-score
    macro_f1 = f1_score(true_labels, predicted_labels, average='macro')

    # Calculate weighted F1-score
    weighted_f1 = f1_score(true_labels, predicted_labels, average='weighted')
    
    speaker_metrics[speaker] = {'accuracy': accuracy, 'macro_f1': macro_f1, 'weighted_f1': weighted_f1}

    print()

# Calculate overall accuracy and macro F1-score
overall_accuracy = final_correct / final
overall_macro_f1 = np.mean([metrics['macro_f1'] for metrics in speaker_metrics.values()])
overall_weighted_f1 = np.mean([metrics['weighted_f1'] for metrics in speaker_metrics.values()])

# print(final_correct)
print(f"Overall Accuracy: {overall_accuracy}")
# print(f"Overall Macro F1-score: {overall_macro_f1}")
# print(f"Overall Weighted F1-score: {overall_weighted_f1}")

Speaker: Mike
Episode: utterance_1666
True Emotions: [7]
Predicted Emotions: [7]
Episode: utterance_1664
True Emotions: [7]
Predicted Emotions: [7]
Episode: utterance_3649
True Emotions: [7, 7, 7, 7]
Predicted Emotions: [7, 7, 7, 7]

Speaker: Kathy
Episode: utterance_1158
True Emotions: [5, 7]
Predicted Emotions: [5, 2]

Speaker: Lauren
Episode: utterance_211
True Emotions: [7, 4]
Predicted Emotions: [7, 4]

Speaker: Leslie
Episode: utterance_701
True Emotions: [2, 7, 5]
Predicted Emotions: [2, 2, 1]
Episode: utterance_699
True Emotions: [2, 7]
Predicted Emotions: [2, 2]

Speaker: an
Episode: utterance_1606
True Emotions: [2, 2]
Predicted Emotions: [2, 2]

Speaker: The Fireman
Episode: utterance_1798
True Emotions: [4, 7]
Predicted Emotions: [4, 7]
Episode: utterance_1797
True Emotions: [4]
Predicted Emotions: [4]

Speaker: Student
Episode: utterance_1429
True Emotions: [7]
Predicted Emotions: [7]
Episode: utterance_1430
True Emotions: [7, 7]
Predicted Emotions: [7, 7]
Episode: utteran